In [1]:
from os.path import join, curdir
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
import cv2

## Vector đặc trưng gương mặt

In [2]:
db_path = join(curdir,"Database", "Database.json")
with open(db_path, "r") as db:
    database = json.load(db)

In [3]:
# i=1
# for label, embedding in database.items():
#     plt.figure(i)
#     for ebd in embedding:
#         plt.plot(ebd)
#     i+=1


## Cân bằng sáng (Histogram Equalization)

In [4]:
test_img = cv2.imread("./Dataset/People/Phuoc/IMG_3263.JPG")
test_img = cv2.resize(test_img, (450, 600))
test_img=cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

# Creating a Histograms Equalization
equ = cv2.equalizeHist(test_img)

# Stacking images side-by-side
res = np.hstack((test_img, equ))

# Show image input vs output
cv2.imshow("", res)
cv2.waitKey(0)
cv2.destroyAllWindows()